<h2>FT006 - Inversiones Depositos a la vista </h2>
<p> En este apartado se trata la base de datos FT006 </p>

In [ ]:
#Cargamos los paquetes que vamos a necesitar
import pandas as pd
import numpy as np
import pyreadstat
pd.pandas.set_option('display.max_columns', None)

In [ ]:
# Se carga el FT006 para 2020 y 2021 completo desde el archivo .dta
FT006_2020_2021 = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/Informe_Pagos/FT006 20202021.dta')

In [ ]:
# Se carga el FT006 para para diciembre de 2019 desde el archivo .dta
FT006_2019 = pd.read_stata('C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/Informe_Pagos/FT006 122019.dta')

In [ ]:
# Se concatenan la bases de datos 
frames = [FT006_2020_2021, FT006_2019]
FT006 = pd.concat(frames, ignore_index=True)

In [ ]:
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria
del [FT006_2020_2021, FT006_2019, frames]

In [ ]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT006.rename(columns={'Nit': 'nit_eps', 'Año': 'ano', 'Periodo': 'mes', 'lineaNegocio': 'linea','claseCuenta': 'clasecuenta', 'saldoExtracto': 'sextracto'}, inplace=True)

In [ ]:
# Se transforman los datos para su correcta identificación
FT006[['nit_eps', 'RazonSocial', 'ano', 'mes', 'linea']] = FT006[['nit_eps', 'RazonSocial', 'ano', 'mes', 'linea']].astype(str)

In [ ]:
# Se filtra la línea de negocio de interés. En este caso corresponde a la línea 1
FT006 = FT006.loc[FT006['linea'] == '1']

In [ ]:
# Se reemplaza la clase de cuenta
FT006.loc[(FT006['clasecuenta'] == 2), 
       'clasecuenta'] = 1 

In [ ]:
# Se encuentran las observaciones que cumplen las condiciones. Se deben eliminar las observaciones con clase de cuenta 3
df_filtered = FT006[
    (FT006['clasecuenta'] == 3)
    ].index

In [ ]:
# Se eliminan las observaciones que cumplen las condiciones anteriore
FT006 = FT006.drop(df_filtered)

In [ ]:
# Se convierte la variable en mes y año
FT006[['ano', 'mes']] = FT006[['ano', 'mes']].astype(float).astype(int)

In [ ]:
# Para hacer la suma de sextracto por clase de cuenta, NIT EPS, NIT proveedor, Mes y Año
FT006 = FT006.groupby(['nit_eps', 'mes', 'ano', 'clasecuenta']).agg({'sextracto': 'sum'}).reset_index()

In [ ]:
# Para hacer la suma de sextracto por NIT EPS, NIT proveedor, Mes y Año
FT006 = FT006.groupby(['nit_eps', 'ano', 'mes'])[['sextracto']].sum().reset_index()

In [ ]:
# Se crea el rango de meses necesarios en cada mes
fecha_ideal=range(1, 12 + 1)
# Se rellena con ceros la información para los meses faltantes
FT006 = FT006.set_index('mes').groupby(['nit_eps', 'ano']).apply(lambda x: x.reindex(index=fecha_ideal, fill_value=0)).drop(['nit_eps', 'ano'], 1)

In [ ]:
# Se desagrupa la información para su posterior filtro pues no nos interesa la información para 2019
FT006 = FT006.reset_index()

In [ ]:
# Se encuentran las observaciones que cumplen las condiciones
df_filtered = FT006[
    (FT006['ano'] == 2019) & (FT006['mes'] == 1) |
    (FT006['ano'] == 2019) & (FT006['mes'] == 2) |
    (FT006['ano'] == 2019) & (FT006['mes'] == 3) |
    (FT006['ano'] == 2019) & (FT006['mes'] == 4) |
    (FT006['ano'] == 2019) & (FT006['mes'] == 5) |
    (FT006['ano'] == 2019) & (FT006['mes'] == 6) |
    (FT006['ano'] == 2019) & (FT006['mes'] == 7) |
    (FT006['ano'] == 2019) & (FT006['mes'] == 8) |
    (FT006['ano'] == 2019) & (FT006['mes'] == 9) |
    (FT006['ano'] == 2019) & (FT006['mes'] == 10) |
    (FT006['ano'] == 2019) & (FT006['mes'] == 11)
    ].index

In [ ]:
# Se eliminan las observaciones que cumplen las condiciones anteriore
FT006 = FT006.drop(df_filtered)

In [ ]:
# Se agrupa la información para calcular su respectiva diferencia mensual por CXP
FT006 = FT006.set_index(['nit_eps', 'ano','mes'])

In [ ]:
# Se calcula la diferencia mensual para conocer el valor mensual de cada cuenta ya que por defecto vienen agregadas
FT006_dif = FT006.diff().fillna(0)

In [ ]:
# Se desagregan las bases de datos para su posterior unión
FT006_dif = FT006_dif.reset_index()
FT006 = FT006.reset_index()

In [ ]:
# Se reemplazan los nombres de las columnas para facilitar el tratamiento
FT006_dif.rename(columns={'sextracto': 'DSaldo'}, inplace=True)

In [ ]:
# Se encuentran las observaciones que cumplen las condiciones
df_filtered = FT006_dif[
    (FT006['ano'] == 2019)
    ].index

In [ ]:
# Se eliminan las observaciones que cumplen las condiciones anteriore
FT006_dif = FT006_dif.drop(df_filtered)

In [ ]:
# Para guardar la base de datos lista para ser tratada en formato Excel y CSV

FT006_dif.to_excel(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/DVFICEPS_2020_2021.xlsx', index = False)
FT006_dif.to_csv(r'C:/Users/Miguel Angel/Documents/Supersalud/Pagos_EPS_Proveedores/DVFICEPS_2020_2021.csv', index = False, encoding='utf-8-sig')

In [ ]:
# Se eliminan la bases que no se necesitan para liberar espacio en la memoria
del [FT006, FT006_dif, df_filtered, fecha_ideal]